In [1]:
! pip install funcy

In [2]:
! pip install gensim
! pip install tzdata
! pip install --no-dependencies pyLDAvis
! pip install wget

In [3]:
# imports
import requests
import spacy
from collections import defaultdict
import wget
from gensim import corpora, models
import pandas as pd
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes('ner', 'parser')

['ner', 'parser']

### Topic Modeling HW

In [5]:
response = requests.get('https://www.gutenberg.org/cache/epub/514/pg514.txt')
text = response.text

In [6]:
text.find('“Christmas won’t be Christmas without any presents,”')

2554

In [7]:
text.find('*** END ')

1028735

In [8]:
start = 2554
end = 1028734

In [9]:
tale = text[start:end]

In [10]:
tale_paras = tale.split('\r\n\r\n')

In [11]:
author = []
title = []

In [12]:
for para in tale_paras:
    author.append('Alcott')
    title.append('Little')

In [13]:
tale_df = pd.DataFrame(list(zip(author, title, tale_paras)), columns=['author', 'title', 'text'])

In [14]:
tale_df.head()

,author,title,text
0,Alcott,Little,“Christmas won’t be Christmas without any pres...
1,Alcott,Little,"“It’s so dreadful to be poor!” sighed Meg, loo..."
2,Alcott,Little,“I don’t think it’s fair for some girls to hav...
3,Alcott,Little,"“We’ve got Father and Mother, and each other,”..."
4,Alcott,Little,The four young faces on which the firelight sh...


In [15]:
def process_text(text):
    """Remove new line characters and lemmatize text. Returns string of lemmas"""
    text = text.replace('\n', ' ')
    doc = nlp(text)
    tokens = [token for token in doc]
    no_stops = [token for token in tokens if not token.is_stop]
    no_punct = [token for token in no_stops if token.is_alpha]
    lemmas = [token.lemma_ for token in no_punct]
    lemmas_lower = [lemma.lower() for lemma in lemmas]
    lemmas_string = ' '.join(lemmas_lower)
    return lemmas_string

In [16]:
tale_df['lemmas'] = tale_df['text'].apply(process_text)

In [17]:
tale_df.head()

,author,title,text,lemmas
0,Alcott,Little,“Christmas won’t be Christmas without any pres...,christmas wo christmas present grumble jo lie rug
1,Alcott,Little,"“It’s so dreadful to be poor!” sighed Meg, loo...",dreadful poor sigh meg look old dress
2,Alcott,Little,“I don’t think it’s fair for some girls to hav...,think fair girl plenty pretty thing girl add l...
3,Alcott,Little,"“We’ve got Father and Mother, and each other,”...",get father mother say beth contentedly corner
4,Alcott,Little,The four young faces on which the firelight sh...,young face firelight shine brighten cheerful w...


In [18]:
length_filter = tale_df['lemmas'].str.len() > 25

In [19]:
tale_df = tale_df[length_filter]

In [20]:
tale_df.head()

,author,title,text,lemmas
0,Alcott,Little,“Christmas won’t be Christmas without any pres...,christmas wo christmas present grumble jo lie rug
1,Alcott,Little,"“It’s so dreadful to be poor!” sighed Meg, loo...",dreadful poor sigh meg look old dress
2,Alcott,Little,“I don’t think it’s fair for some girls to hav...,think fair girl plenty pretty thing girl add l...
3,Alcott,Little,"“We’ve got Father and Mother, and each other,”...",get father mother say beth contentedly corner
4,Alcott,Little,The four young faces on which the firelight sh...,young face firelight shine brighten cheerful w...


In [21]:
def remove_new_lines(text):
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    return text

In [22]:
tale_df['text'] = tale_df['text'].apply(remove_new_lines)

In [23]:
# save our work
tale_df.to_csv('littwomen_novel.csv', index=False)

In [25]:
file_name = 'littwomen_novel.csv'
df = pd.read_csv(file_name)

In [26]:
docs = df['lemmas'].to_list()

In [28]:
len(docs[0])

49

In [29]:
texts =[
    [word for word in doc.lower().split()]
    for doc in docs
]

In [30]:
frequency = defaultdict(int)
for text in texts:
  for token in text:
    frequency[token] += 1

In [31]:
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

In [32]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20, passes=50)
lda_model.print_topics()

[(0,
  '0.035*"laurie" + 0.031*"say" + 0.026*"jo" + 0.023*"look" + 0.020*"hand" + 0.019*"shake" + 0.018*"away" + 0.013*"eye" + 0.012*"bow" + 0.011*"begin"'),
 (1,
  '0.050*"say" + 0.031*"jo" + 0.028*"yes" + 0.020*"add" + 0.019*"oh" + 0.018*"laurie" + 0.015*"shall" + 0.013*"look" + 0.013*"amy" + 0.012*"sort"'),
 (2,
  '0.018*"old" + 0.017*"jo" + 0.014*"hand" + 0.012*"forget" + 0.012*"look" + 0.010*"gentleman" + 0.010*"take" + 0.008*"laugh" + 0.008*"man" + 0.008*"like"'),
 (3,
  '0.040*"jo" + 0.023*"beth" + 0.023*"come" + 0.015*"look" + 0.014*"face" + 0.014*"amy" + 0.013*"say" + 0.013*"laurie" + 0.011*"go" + 0.011*"away"'),
 (4,
  '0.018*"meg" + 0.014*"little" + 0.013*"go" + 0.012*"dress" + 0.011*"time" + 0.010*"amy" + 0.008*"get" + 0.008*"look" + 0.007*"laugh" + 0.007*"white"'),
 (5,
  '0.024*"day" + 0.017*"sit" + 0.013*"work" + 0.012*"long" + 0.011*"little" + 0.008*"year" + 0.008*"sweet" + 0.008*"play" + 0.008*"amy" + 0.008*"give"'),
 (6,
  '0.015*"good" + 0.014*"man" + 0.013*"old" + 0

In [35]:
lda_model.get_document_topics(corpus[0])

[(3, 0.5118977), (13, 0.3880128)]

In [36]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.214014 -0.061537       1        1  14.104515
3     -0.067738 -0.088398       2        1  13.199091
4     -0.001894  0.020866       3        1   7.264153
19    -0.049216 -0.131118       4        1   6.507092
8     -0.214796  0.103172       5        1   6.227675
7     -0.042037 -0.084999       6        1   5.795749
14     0.005514  0.070230       7        1   5.557682
17    -0.046195  0.034792       8        1   4.318312
6     -0.051744  0.052216       9        1   4.270678
2      0.068669  0.005134      10        1   4.030474
5     -0.118586  0.121380      11        1   3.964197
11     0.085620  0.035974      12        1   3.780341
15    -0.003799 -0.093330      13        1   3.533322
12     0.073852  0.017027      14        1   3.206069
1      0.022424 -0.141708      15        1   3.117164
0      0.136004 -0.106791      16        1   2.893578
13     0.050454  0.101365      17        1   2.703872
16     0.122931 -0.143873      18        1   2.097688
10     0.142837  0.172784      19        1   2.073616
18     0.101714  0.116813      20        1   1.354734, topic_info=      Term         Freq        Total Category  logprob  loglift
31     say   863.000000   863.000000  Default  30.0000  30.0000
2       jo  1265.000000  1265.000000  Default  29.0000  29.0000
122   like   670.000000   670.000000  Default  28.0000  28.0000
221  march   277.000000   277.000000  Default  27.0000  27.0000
9     look   666.000000   666.000000  Default  26.0000  26.0000
..     ...          ...          ...      ...      ...      ...
328   meet     6.020311    52.684372  Topic20  -5.1139   2.1324
2       jo     8.151482  1265.176604  Topic20  -4.8109  -0.7432
14     add     6.250231    97.928326  Topic20  -5.0764   1.5499
95     cry     6.144590   180.781445  Topic20  -5.0935   0.9198
78    good     5.727922   598.349027  Topic20  -5.1637  -0.3473

[1471 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3365      3  0.407155      able
3365      8  0.116330      able
3365     19  0.407155      able
3395      3  0.868729   absence
3213      7  0.852991  absently
...     ...       ...       ...
1391      8  0.142919     youth
1391      9  0.285838     youth
1391     10  0.476397     youth
3967      5  0.910299  youthful
1054     19  0.902361      zara

[3794 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 5, 20, 9, 8, 15, 18, 7, 3, 6, 12, 16, 13, 2, 1, 14, 17, 11, 19])

In [37]:
## TF idf topic model
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
lda_model_tfidf = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=20, passes=50)
corpus_lda = lda_model_tfidf[corpus_tfidf]
lda_model_tfidf.print_topics()

[(0,
  '0.008*"silk" + 0.007*"somewhat" + 0.007*"lace" + 0.007*"tall" + 0.007*"improve" + 0.007*"likewise" + 0.006*"tuck" + 0.006*"lend" + 0.006*"tree" + 0.006*"flatter"'),
 (1,
  '0.008*"embrace" + 0.007*"implore" + 0.007*"mixture" + 0.007*"allude" + 0.007*"correct" + 0.007*"merriment" + 0.007*"vent" + 0.006*"manuscript" + 0.006*"forlorn" + 0.006*"impertinent"'),
 (2,
  '0.007*"fail" + 0.006*"lovely" + 0.006*"pleasant" + 0.006*"usual" + 0.006*"place" + 0.005*"drive" + 0.005*"month" + 0.005*"letter" + 0.005*"hang" + 0.005*"tongue"'),
 (3,
  '0.008*"laurie" + 0.008*"ask" + 0.008*"jo" + 0.007*"say" + 0.006*"look" + 0.006*"stop" + 0.006*"laugh" + 0.006*"amy" + 0.006*"think" + 0.006*"smile"'),
 (4,
  '0.008*"number" + 0.007*"arrangement" + 0.007*"particle" + 0.007*"rash" + 0.007*"fist" + 0.006*"defend" + 0.006*"blind" + 0.006*"worn" + 0.006*"ghost" + 0.006*"amusement"'),
 (5,
  '0.013*"lamb" + 0.010*"suggest" + 0.010*"attempt" + 0.008*"immensely" + 0.007*"blunt" + 0.007*"meaning" + 0.007*"

In [38]:
lda_model_tfidf.get_document_topics(corpus_tfidf[0])

[(0, 0.014893395),
 (1, 0.014893395),
 (2, 0.014893395),
 (3, 0.014893395),
 (4, 0.014893395),
 (5, 0.014893395),
 (6, 0.014893395),
 (7, 0.014893395),
 (8, 0.014893395),
 (9, 0.14615186),
 (10, 0.014893395),
 (11, 0.45020485),
 (12, 0.014893395),
 (13, 0.014893395),
 (14, 0.15045558),
 (15, 0.014893395),
 (16, 0.014893395),
 (17, 0.014893395),
 (18, 0.014893395),
 (19, 0.014893395)]

In [39]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.404376 -0.073942       1        1  38.874502
3      0.281622  0.112705       2        1  12.668954
2     -0.041396  0.198962       3        1   6.066436
12    -0.052817  0.110309       4        1   5.122207
14    -0.034965  0.106089       5        1   4.311437
10    -0.044809  0.047932       6        1   3.752959
19    -0.045417 -0.011858       7        1   2.996153
0     -0.048162 -0.024408       8        1   2.814655
15    -0.024135 -0.028991       9        1   2.549735
17    -0.038817 -0.042440      10        1   2.210309
9     -0.034447 -0.051534      11        1   2.147588
6     -0.038504 -0.035384      12        1   2.141367
13    -0.039937 -0.040632      13        1   1.967652
18    -0.040118 -0.040093      14        1   1.942967
4     -0.034772 -0.045832      15        1   1.812702
16    -0.033226 -0.037900      16        1   1.785939
8     -0.035099 -0.038345      17        1   1.780754
7     -0.034812 -0.032498      18        1   1.771798
5     -0.034358 -0.033875      19        1   1.741057
1     -0.030208 -0.038263      20        1   1.540829, topic_info=             Term       Freq      Total Category  logprob  loglift
3857    professor  12.000000  12.000000  Default  30.0000  30.0000
1804      chapter   7.000000   7.000000  Default  29.0000  29.0000
1403       laurie  47.000000  47.000000  Default  28.0000  28.0000
2              jo  62.000000  62.000000  Default  27.0000  27.0000
256           ask  40.000000  40.000000  Default  26.0000  26.0000
...           ...        ...        ...      ...      ...      ...
3679      justice   0.632656   1.309134  Topic20  -5.7637   3.4457
1469  reluctantly   0.990276   2.075032  Topic20  -5.3156   3.4331
3926   prediction   0.607867   1.284345  Topic20  -5.8037   3.4248
1520      freedom   0.704325   1.601921  Topic20  -5.6564   3.3511
2794         snub   0.696507   1.593844  Topic20  -5.6675   3.3450

[922 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
836      11  0.460552       abash
3365      7  0.815222        able
2401     19  0.674853  abominable
3395     15  0.545989     absence
3213     16  0.904075    absently
...     ...       ...         ...
497       6  0.068911       write
750       3  0.132668       wrong
750       7  0.132668       wrong
750      12  0.530672       wrong
750      14  0.132668       wrong

[1270 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 4, 3, 13, 15, 11, 20, 1, 16, 18, 10, 7, 14, 19, 5, 17, 9, 8, 6, 2])